In [53]:
import pandas as pd
import numpy as np
departments = pd.read_csv('departments.csv', sep=',', encoding="ISO-8859-1")
products = pd.read_csv('products.csv', sep=',', encoding="ISO-8859-1")
mergedOrders = pd.read_csv('mergedOrders.csv', sep=',', encoding="ISO-8859-1")
orders1 = pd.read_csv('orders.csv', sep=',', encoding="ISO-8859-1")

In [54]:
products.head()
departments.head()
mergedOrders.head()

,Unnamed: 0,order_id,product_id,reordered,user_id,days_since_prior_order
0,0,8,23423,1,3107,17.0
1,1,40,10070,1,382,23.0
2,2,40,42450,1,382,23.0
3,3,40,33198,1,382,23.0
4,4,40,34866,1,382,23.0


In [55]:
#mergedOrders = mergedOrders.drop(columns = ["order_id"])
dfDaysAverage = mergedOrders.groupby("product_id").mean()
dfDaysAverage.sort_values("days_since_prior_order").dropna()
dfDaysAverage = dfDaysAverage.drop(columns=["Unnamed: 0","order_id","reordered","user_id"])
dfDaysAverage = dfDaysAverage.rename(columns={"days_since_prior_order": "mediaEcompras"})
dfDaysAverage = dfDaysAverage[dfDaysAverage["mediaEcompras"] > 0]
dfDaysAverage['product_id'] = dfDaysAverage.index
dfDaysAverage.head()

,mediaEcompras,product_id
product_id,,
1,12.333333,1
2,20.500000,2
3,19.250000,3
4,12.000000,4
7,10.666667,7


In [56]:
dfQuantity = mergedOrders.groupby("product_id").count()
dfQuantity = dfQuantity.drop(columns=["Unnamed: 0","order_id","reordered","days_since_prior_order"]).reset_index()
dfQuantity = dfQuantity.rename(columns={"user_id": "cantitatMitjaProductes"})
dfQuantity = dfQuantity.sort_values('cantitatMitjaProductes')
dfQuantity['product_id'] = dfQuantity.index
dfQuantity.head()

,product_id,cantitatMitjaProductes
20144,20144,1
28037,28037,1
13241,13241,1
13242,13242,1
13243,13243,1


In [57]:
allMergedOrders = mergedOrders
aux = allMergedOrders.groupby( [ "user_id", "product_id"] ).size().to_frame(name = 'total').reset_index()
allMergedOrders = allMergedOrders.groupby( [ "product_id", "user_id"] ).mean().reset_index()
allMergedOrders = allMergedOrders.drop(columns=["Unnamed: 0","order_id","reordered"]).reset_index()
allMergedOrders = pd.merge(aux,allMergedOrders, on=[ "user_id", "product_id"])
allMergedOrders = allMergedOrders.rename(columns={"days_since_prior_order": "media_usuarioEcompras"})
allMergedOrders = allMergedOrders.drop(columns=["index"])
allMergedOrders.head()

,user_id,product_id,total,media_usuarioEcompras
0,1,196,11,19.000000
1,1,10258,10,19.000000
2,1,10326,1,28.000000
3,1,12427,10,19.555556
4,1,13032,4,19.750000


In [103]:
def puntTemps(df):
    a2 = df['mediaEcompras'] / 2

    if df['media_usuarioEcompras'] <= df['mediaEcompras']: 
        return 50
    elif df['media_usuarioEcompras'] < df['mediaEcompras'] + a2:
        return 22.5
    else: 
        return 12.5

def puntQuant(df):
    a2 = df['cantitatMitjaProductes'] / 2
    
    if df['total'] < a2: 
        return 12.5
    elif df['total'] < df['cantitatMitjaProductes']:
        return 22.5
    else: 
        return 50

In [105]:
dfDaysAverage.index.name = None
dfProva = allMergedOrders
#dfProva = dfProva.groupby(["user_id",'product_id']).agg({'product_id':['mean', 'median']})
#dfProva['puntuacio'] = dfProva.age * dfProva.bmi
#dfProva = dfProva.groupby( [ "user_id", "product_id"] ).size().to_frame(name = 'total').reset_index()
dfProva =  pd.merge(dfProva, dfQuantity, on="product_id")
dfProva =  pd.merge(dfProva, dfDaysAverage, on="product_id")
dfProva['puntCant'] = dfProva.apply(puntQuant,axis=1)
dfProva['punTemps'] = dfProva.apply(puntTemps,axis=1)
dfProva['punTotal'] = dfProva.punTemps + dfProva.puntCant
dfProva


,user_id,product_id,total,media_usuarioEcompras,cantitatMitjaProductes,mediaEcompras,puntCant,punTemps,punTotal
0,1,196,11,19.000000,9,12.160839,50.0,12.5,62.5
1,15,196,5,8.600000,9,12.160839,22.5,50.0,72.5
2,19,196,3,6.666667,9,12.160839,12.5,50.0,62.5
3,21,196,1,6.000000,9,12.160839,12.5,50.0,62.5
4,31,196,2,7.500000,9,12.160839,12.5,50.0,62.5
...,...,...,...,...,...,...,...,...,...
245292,6038,4078,1,5.000000,39,5.000000,12.5,50.0,62.5
245293,6038,19526,1,15.000000,2,15.000000,22.5,50.0,72.5
245294,6038,20096,2,7.500000,1,7.500000,50.0,50.0,100.0
245295,6043,21628,1,30.000000,4,30.000000,12.5,50.0,62.5


In [106]:
dfTotal = dfQuantity
dfTotal = pd.merge(dfDaysAverage, dfTotal, on = "product_id")

In [63]:
dfTotal = dfTotal.sort_values('cantitatMitjaProductes')
dfTotal

,mediaEcompras,product_id,cantitatMitjaProductes
12369,9.125000,20334,1
11605,13.500000,19025,1
11611,26.000000,19032,1
11612,11.600000,19033,1
4279,20.666667,7046,1
...,...,...,...
18280,15.000000,29910,5065
18054,14.335227,29553,6813
8307,30.000000,13695,7115
7992,23.000000,13191,8182


In [ ]:
dfPuntUsuariTotal = dfProva.drop(columns=["Unnamed: 0","order_id","reordered"])